In [24]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated,Any,Dict
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()

openrouter_key = os.getenv("OPENROUTER_API_KEY")
if not openrouter_key:
    raise ValueError("OPENROUTER_API_KEY not found in environment")

# Initialize ChatOpenAI with OpenRouter
llm = ChatOpenAI(
    model="gpt-5",            
    temperature=0,
    max_tokens=32768,         
    openai_api_key=openrouter_key,
    base_url="https://openrouter.ai/api/v1"
)


In [26]:
class Text_Summarization(TypedDict):
    md_files:str
    text:Dict[str,str]
    llm_output:str


In [27]:
def text_extractor(state: Text_Summarization) -> Dict[str, Dict[str, str]]:
    folder_path = state["md_files"]
    md_content: Dict[str, str] = {}

    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                md_content[filename] = content

    return {"text": md_content}

def llm_summraizer(state: Text_Summarization) -> dict:
    # Combine all md file contents into a single string
    all_content = "".join(state["text"].values())

    output_path = r"D:\New folder\LangGraph\output_readme.md"  # raw string and .md extension

    messages = [
        SystemMessage(content="You are a text summarizer which summarizes text very keenly without missing any point."),
        HumanMessage(content=f"This is a vulnerability assessment report found by multiple penetration testing agents. Summarize it properly:\n{all_content}")
    ]

    summarizer = llm.invoke(messages)  # returns AIMessage object
    summary_text = summarizer.content

    # Save summary to Markdown file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(summary_text)

    # Return the summary as dict
    return {"llm_output": summary_text}

In [28]:
graph = StateGraph(Text_Summarization)
graph.add_node("text_extractor",text_extractor)
graph.add_node("llm_summraizer",llm_summraizer)

graph.add_edge(START,"text_extractor")
graph.add_edge("text_extractor","llm_summraizer")
graph.add_edge("llm_summraizer",END)

workflow = graph.compile()
workflow.invoke({"md_files": r"D:\New folder\LangGraph\Md_files"})


{'md_files': 'D:\\New folder\\LangGraph\\Md_files',
 'text': {'vuln-0001.md': '# Reflected XSS via pageLanguage parameter on https://boxxinsurance.com/ca/\n\n**ID:** vuln-0001\n**Severity:** CRITICAL\n**Found:** 2025-12-25 11:41:41 UTC\n\n## Description\n\nEndpoint: https://boxxinsurance.com/ca/?pageLanguage=PAYLOAD  \nMethod: GET  \nVulnerable parameter: pageLanguage  \nPayload used: <svg/onload=alert(1)>  \nVerification: The payload was reflected in the response and executed in the browser, confirmed by console log “XSS triggered”.  \nImpact: Arbitrary JavaScript execution leads to session hijacking, credential theft, and further attacks.  \nRemediation: Proper output encoding based on HTML context, strict Content‑Security‑Policy (disallow unsafe‑inline), and input validation/whitelisting.  \nReferences: OWASP XSS Prevention Cheat Sheet.  \nCVSS v3.1: 9.8 (Critical)\n',
  'vuln-0002.md': '# Reflected XSS via nonce parameter on https://boxxinsurance.com/ca/\n\n**ID:** vuln-0002\n**Sev